In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import gpytorch
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.stats import qmc

/home/cadmin/anaconda3/envs/Data_Science_V2/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
# Set device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the Berger Viscous Equation parameters
c = 1.0    # Wave speed
mu = 0.1   # Viscosity coefficient
lam = 1.0  # Nonlinearity coefficient

# Define the neural network
class PINN(nn.Module):
    def __init__(self, layers):
        super(PINN, self).__init__()
        self.net = nn.Sequential(*[
            nn.Sequential(nn.Linear(layers[i], layers[i+1]), nn.Tanh())
            for i in range(len(layers)-2)
        ] + [nn.Linear(layers[-2], layers[-1])])

    def forward(self, x, t):
        inputs = torch.cat((x, t), dim=1)
        return self.net(inputs)

def generate_collocation_points(N_f, L=1.0, T=1.0):
    # Use Latin Hypercube Sampling for better distribution
    sampler = qmc.LatinHypercube(d=2)  # 2D (x, t) space
    sample = sampler.random(N_f)  # Generate N_f samples in [0, 1]^2

    # Scale samples: x in [-L, L], t in [0, T]
    x_f = torch.tensor((sample[:, 0] * 2 - 1) * L, dtype=torch.float32).reshape(-1, 1)
    t_f = torch.tensor(sample[:, 1] * T, dtype=torch.float32).reshape(-1, 1)

    return x_f.to(device), t_f.to(device)
def compute_pde_residual(model, x, t):
    x = x.clone().detach().requires_grad_(True)
    t = t.clone().detach().requires_grad_(True)

    u = model(x, t)  # Predict u(x, t)

    u_t = torch.autograd.grad(u, t, torch.ones_like(u), create_graph=True)[0]
    u_x = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]

    # Inviscid Burgers' equation: u_t + u * u_x = 0
    f = u_t + u * u_x
    return f


# Define loss function
def loss_function(model, x_f, t_f, x_bc, t_bc, u_bc):
    f_residual = compute_pde_residual(model, x_f, t_f)
    loss_pde = torch.mean(f_residual**2)

    u_pred_bc = model(x_bc, t_bc)
    loss_bc = torch.mean((u_pred_bc - u_bc)**2)

    return loss_pde + loss_bc

def val_loss(model, x_f, t_f, x_bc, t_bc, u_bc):
    f_residual = compute_pde_residual(model, x_f, t_f)
    loss_pde = torch.mean(f_residual**2)

    u_pred_bc = model(x_bc, t_bc)
    loss_bc = torch.mean((u_pred_bc - u_bc)**2)

    return loss_pde + loss_bc

# Training loop
def train(model, optimizer, N_f, x_f, t_f, valX, valT, x_bc, t_bc, u_bc, epochs=5000,threshold = 0.001):
    val_scores = []
    thresh_e = epochs
    for epoch in range(epochs):
        optimizer.zero_grad()
        loss = loss_function(model, x_f, t_f, x_bc, t_bc, u_bc)
        loss.backward()
        optimizer.step()

        loss_val = val_loss(model, valX, valT, x_bc, t_bc, u_bc)

        if loss_val.item() < threshold and thresh_e >= epochs:
            print("Threshold reach at:",epoch)
            print("Val loss:",loss_val)
            thresh_e = epoch

        if epoch % 500 == 0 or epoch == epochs-1:
            print(f"Epoch {epoch}, Loss: {loss.item()}")
            val_scores.append(loss_val.item())
        x_f,t_f = generate_collocation_points(N_f)

    return thresh_e, val_scores

In [3]:

def gen_points_import(model, N_f, L=1.0, T=1.0):
    x_f = (torch.rand(N_f, 1, device=device, requires_grad=True) * 2 - 1) * L  # x in [-L, L]
    t_f = torch.rand(N_f, 1, device=device, requires_grad=True) * T  # t in [0, T]
    
    if model is not None:  # Perform importance sampling
        residuals = compute_pde_residual(model, x_f, t_f).detach()
        probabilities = residuals.abs() / torch.sum(residuals.abs())
        sampled_indices = torch.multinomial(probabilities.view(-1), N_f, replacement=True)
        x_f, t_f = x_f[sampled_indices], t_f[sampled_indices]
    
    return x_f, t_f

def train_import(model, optimizer, N_f, x_f,t_f,valX ,valT ,x_bc, t_bc, u_bc, epochs=10000,threshold = 0.001):

    val_scores = []
    thresh_e = epochs
    for epoch in range(epochs):
        # if epoch % resample_every == 0 and epoch >=500:
        #     x_f, t_f = gen_points_import(model, N_f)
        
        optimizer.zero_grad()
        loss = loss_function(model, x_f, t_f, x_bc, t_bc, u_bc)
        loss.backward()
        optimizer.step()

        loss_val = val_loss(model, valX, valT, x_bc, t_bc, u_bc)

        if loss_val.item() < threshold and thresh_e >= epochs:
            print("Threshold reach at:",epoch)
            print("Val loss:",loss_val)
            thresh_e = epoch
        if epoch % 500 == 0 or epoch == epochs-1:
            print(f"Epoch {epoch}, Loss: {loss.item()}")
            val_scores.append(loss_val.item())
        x_f, t_f = gen_points_import(model, N_f)
    return thresh_e, val_scores

In [4]:
# Gaussian Process Model for Importance Sampling
class ResidualGP(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ResidualGP, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        # self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        self.covar_module = gpytorch.kernels.ScaleKernel(
    gpytorch.kernels.MaternKernel(nu=1.5)
)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [5]:
import torch
import gpytorch
from scipy.stats import qmc

def generate_collocation_points_with_gp(model, N_f, x_f, t_f, x_bc=None, t_bc=None, u_bc=None, 
                                        L=1.0, T=1.0, alpha=0.5, fraction_gp=0.5, residual_thresh=1e-3):
    device = x_f.device

    # === Step 1: Prepare GP training data ===
    x_f,t_f = gen_points_import(model, N_f)
    x_train = x_f
    t_train = t_f
    xt_train = torch.cat([x_train, t_train], dim=1).detach()

    if model is not None:
        with torch.no_grad():
            u_train = model(x_train, t_train).detach().view(-1)
            xt_all = xt_train
            u_all = u_train

            if x_bc is not None and t_bc is not None and u_bc is not None:
                xt_bc = torch.cat([x_bc, t_bc], dim=1).detach()
                u_bc = u_bc.detach().view(-1)
                xt_all = torch.cat([xt_all, xt_bc], dim=0)
                u_all = torch.cat([u_all, u_bc], dim=0)

        # === Train GP ===
        likelihood = gpytorch.likelihoods.GaussianLikelihood()
        gp_model = ResidualGP(xt_all, u_all, likelihood).to(device)

        gp_model.train()
        likelihood.train()
        optimizer = torch.optim.Adam(gp_model.parameters(), lr=0.01)
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, gp_model)

        for _ in range(100):
            optimizer.zero_grad()
            output = gp_model(xt_all)
            loss = -mll(output, u_all)
            loss.backward()
            optimizer.step()

        gp_model.eval()
        likelihood.eval()

        # === Step 2: Generate candidate points ===
        sampler = qmc.LatinHypercube(d=2)
        sample = sampler.random(10 * N_f)
        x_cand = torch.tensor(sample[:, 0] * L, dtype=torch.float32).reshape(-1, 1).to(device)
        t_cand = torch.tensor(sample[:, 1] * T, dtype=torch.float32).reshape(-1, 1).to(device)
        xt_cand = torch.cat([x_cand, t_cand], dim=1)

        # === Step 3: Sample from GP posterior ===
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            dist = gp_model(xt_cand)
            gp_samples = dist.rsample(torch.Size([1])) # shape: [1, N_cand]
            gp_sample_abs = gp_samples.squeeze(0).abs().detach()  # [N_cand]

        # === Step 4: Compute PDE residuals ===
        x_cand.requires_grad_()
        t_cand.requires_grad_()
        residual = compute_pde_residual(model, x_cand, t_cand).detach().abs().view(-1)

        # === Step 5: Normalize and combine scores ===
        residual = torch.where(residual < residual_thresh, torch.tensor(0.0, device=device), residual)

        sample_score = gp_sample_abs / (gp_sample_abs.sum() + 1e-8)
        residual_score = residual / (residual.sum() + 1e-8)

        sampling_score = alpha * sample_score + (1 - alpha) * residual_score
        sampling_score = sampling_score / (sampling_score.sum() + 1e-8)

        # === Step 6: Hybrid sampling ===
        N_gp = int(fraction_gp * N_f)
        N_rand = N_f - N_gp

        sampled_indices_gp = torch.multinomial(sampling_score, N_gp, replacement=False)
        sampled_indices_rand = torch.randint(0, len(x_cand), (N_rand,), device=device)
        sampled_indices = torch.cat([sampled_indices_gp, sampled_indices_rand], dim=0)

        x_f_new = x_cand[sampled_indices].detach().clone().requires_grad_()
        t_f_new = t_cand[sampled_indices].detach().clone().requires_grad_()
        uncertainty_top = gp_sample_abs[sampled_indices].detach().cpu()

    else:
        x_f_new = x_f.clone().detach().requires_grad_()
        t_f_new = t_f.clone().detach().requires_grad_()
        uncertainty_top = None
        gp_model = None

    return x_f_new, t_f_new, uncertainty_top, gp_model


In [6]:
def train_GP(model, optimizer, N_f, x_f,t_f,valX, valT,x_bc, t_bc, u_bc, epochs=5000, resample_every=500,threshold = 0.001):
    thresh_e = epochs
    val_scores = []
    for epoch in range(epochs):
        # if epoch % resample_every == 0 and epoch >=500:
        #     x_f, t_f = gen_points_import(model, N_f)
        
        optimizer.zero_grad()
        loss = loss_function(model, x_f, t_f, x_bc, t_bc, u_bc)
        loss.backward()
        optimizer.step()

        loss_val = val_loss(model, valX, valT, x_bc, t_bc, u_bc)

        if loss_val.item() < threshold and thresh_e >= epochs:
            print("Threshold reach at:",epoch)
            print("Val loss:",loss_val)
            thresh_e = epoch

        if epoch % 500 == 0 or epoch == epochs-1:
            # x_f,t_f = fit_GP(x_f,t_f)
            print(f"Epoch {epoch}, Loss: {loss.item()}")
            val_scores.append(loss_val.item())
            # x_uncertain, t_uncertain,x,g = generate_collocation_points_with_gp(model,N_f,x_f, t_f) 
        if epoch % resample_every == 0 and epoch > 0 :
            # x_f, t_f = gen_points_import(model, N_f)
            x_uncertain, t_uncertain,uncertainties,gp_model = generate_collocation_points_with_gp(model,N_f,x_f, t_f,x_bc,t_bc,u_bc)

            x_f = x_uncertain
            t_f = t_uncertain

    return thresh_e, val_scores

In [7]:
import torch
import gpytorch
from scipy.stats import qmc

def generate_collocation_points_with_gp_res(model, N_f, x_f, t_f, x_bc=None, t_bc=None, u_bc=None, 
                                            L=2.0, T=1.0, alpha=0.5, fraction_gp=0.5, residual_thresh=1e-3):

    device = x_f.device
    # x_f,t_f = generate_collocation_points(N_f)
    x_f,t_f = gen_points_import(model, N_f)
    x_train = x_f
    t_train = t_f
    xt_train = torch.cat([x_train, t_train], dim=1).detach()

    if model is not None:
        # ✅ Compute residuals (no torch.no_grad here)
        residual_train = compute_pde_residual(model, x_train.requires_grad_(), t_train.requires_grad_()).detach().view(-1)

        xt_all = xt_train
        residual_all = residual_train

        if x_bc is not None and t_bc is not None and u_bc is not None:
            xt_bc = torch.cat([x_bc, t_bc], dim=1).detach()
            residual_bc = compute_pde_residual(model, x_bc.requires_grad_(), t_bc.requires_grad_()).detach().view(-1)

            xt_all = torch.cat([xt_all, xt_bc], dim=0)
            residual_all = torch.cat([residual_all, residual_bc], dim=0)

        # === Train GP on residuals ===
        likelihood = gpytorch.likelihoods.GaussianLikelihood()
        gp_model = ResidualGP(xt_all, residual_all, likelihood).to(device)

        gp_model.train()
        likelihood.train()
        optimizer = torch.optim.Adam(gp_model.parameters(), lr=0.01)
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, gp_model)

        for _ in range(100):
            optimizer.zero_grad()
            output = gp_model(xt_all)
            loss = -mll(output, residual_all)
            loss.backward()
            optimizer.step()

        gp_model.eval()
        likelihood.eval()

        # === Step 2: Candidate points ===
        sampler = qmc.LatinHypercube(d=2)
        sample = sampler.random(10 * N_f)

        # x ∈ [-L/2, L/2], t ∈ [0, T]
        x_cand = torch.tensor((sample[:, 0] * 2 - 1) * (L/2), dtype=torch.float32, device=device).reshape(-1, 1)
        t_cand = torch.tensor(sample[:, 1] * T, dtype=torch.float32, device=device).reshape(-1, 1)
        xt_cand = torch.cat([x_cand, t_cand], dim=1)

        # === Step 3 (Modified): Sample from GP posterior ===
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            dist = gp_model(xt_cand)
            gp_samples = dist.rsample(torch.Size([1]))  # [1, N_cand]
            gp_sample_abs = gp_samples.squeeze(0).abs().detach()  # [N_cand]
  # [N_cand]

        # === Step 4: Compute PDE residuals at candidate points
        x_cand.requires_grad_()
        t_cand.requires_grad_()
        residual = compute_pde_residual(model, x_cand, t_cand).detach().abs().view(-1)

        # === Step 5: Normalize and threshold ===
        residual = torch.where(residual < residual_thresh, torch.tensor(0.0, device=device), residual)

        sample_score = gp_sample_abs / (gp_sample_abs.sum() + 1e-8)
        residual_score = residual / (residual.sum() + 1e-8)

        sampling_score = alpha * sample_score + (1 - alpha) * residual_score
        sampling_score = sampling_score / (sampling_score.sum() + 1e-8)

        # === Step 6: Hybrid sampling ===
        N_gp = int(fraction_gp * N_f)
        N_rand = N_f - N_gp

        sampled_indices_gp = torch.multinomial(sampling_score, N_gp, replacement=False)
        sampled_indices_rand = torch.randint(0, len(x_cand), (N_rand,), device=device)
        sampled_indices = torch.cat([sampled_indices_gp, sampled_indices_rand], dim=0)

        x_f_new = x_cand[sampled_indices].detach().clone().requires_grad_()
        t_f_new = t_cand[sampled_indices].detach().clone().requires_grad_()
        uncertainty_top = gp_sample_abs[sampled_indices].detach().cpu()

    else:
        x_f_new = x_f.clone().detach().requires_grad_()
        t_f_new = t_f.clone().detach().requires_grad_()
        uncertainty_top = None
        gp_model = None

    return x_f_new, t_f_new, uncertainty_top, gp_model


In [8]:
def train_GP_res(model, optimizer, N_f, x_f,t_f,valX, valT,x_bc, t_bc, u_bc, epochs=5000, resample_every=500,threshold = 0.001):
    val_scores = []
    thresh_e = epochs
    for epoch in range(epochs):        
        optimizer.zero_grad()
        loss = loss_function(model, x_f, t_f, x_bc, t_bc, u_bc)
        loss.backward()
        optimizer.step()

        loss_val = val_loss(model, valX, valT, x_bc, t_bc, u_bc)

        if loss_val.item() < threshold and thresh_e >= epochs:
            print("Threshold reach at:",epoch)
            print("Val loss:",loss_val)
            thresh_e = epoch

        if epoch % 500 == 0 or epoch == epochs-1:
            # x_f,t_f = fit_GP(x_f,t_f)
            print(f"Epoch {epoch}, Loss: {loss.item()}")
            val_scores.append(loss_val.item())
            # x_uncertain, t_uncertain,x,g = generate_collocation_points_with_gp(model,N_f,x_f, t_f) 
        if epoch % resample_every == 0 and epoch > 0 :
            x_f, t_f = gen_points_import(model, N_f)
            x_uncertain, t_uncertain,uncertainties,gp_model = generate_collocation_points_with_gp_res(model,N_f,x_f, t_f,x_bc,t_bc,u_bc)

            x_f = x_uncertain
            t_f = t_uncertain

    return thresh_e,val_scores

In [9]:
def compute_pde_residual(model, x, t):
    x.requires_grad = True
    t.requires_grad = True

    u = model(x, t)  # Predict u(x, t)

    u_t = torch.autograd.grad(u, t, torch.ones_like(u), create_graph=True)[0]
    u_x = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]

    # Inviscid Burgers' equation: u_t + u * u_x = 0
    f = u_t + u * u_x
    return f

In [10]:
def compute_pde_residual(model, x, t):
    x = x.detach().requires_grad_()
    t = t.detach().requires_grad_()

    u = model(x, t)  # Predict u(x, t)

    # Compute ∂u/∂t
    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]

    # Compute ∂u/∂x
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]

    # Compute ∂²u/∂x²
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), retain_graph=True, create_graph=True)[0]

    # Burgers' equation residual: u_t + u * u_x - ν * u_xx
    nu = 0.01 / np.pi
    # residual = u_t + u * u_x - nu * u_xx
    residual = u_t + u * u_x 
    return residual


In [11]:
#different initial/boundary conditions
def initial_condition(x, condition_type="sin"):
    if condition_type == "sin":
        return (torch.sin(np.pi * x.cpu())).to(device)
    elif condition_type == "gaussian":
        # return torch.exp(-10 * (x - 0.5) ** 2).to(device)
        mu=0.5
        sigma=0.1
        return torch.exp(-((x - mu)**2) / (2 * sigma**2))
    elif condition_type == "step":
        return torch.where(x < 0.5, torch.tensor(1.0, device=device), torch.tensor(0.0, device=device))
    else:
        raise ValueError("Unknown initial condition type")

def boundary_condition(x, t, boundary_type="dirichlet"):
    if boundary_type == "dirichlet":
        return torch.zeros_like(x).to(device)  # u(0,t) = 0, u(L,t) = 0
    elif boundary_type == "neumann":
        return torch.autograd.grad(model(x, t), x, torch.ones_like(x), create_graph=True)[0]
    elif boundary_type == "periodic":
        return model(x, t) - model(x + L, t)  # u(0,t) = u(L,t)
    else:
        raise ValueError("Unknown boundary condition type")


In [12]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import gc
import torch
import numpy as np
import matplotlib.pyplot as plt

layers = [2, 50, 50, 50, 1]
N_f = 1000
L, T = 1.0, 1.0
N_bc = 100
epochs = 5000
threshold = 0.001

x_f, t_f = generate_collocation_points(N_f, L, T)
x_val, t_val = generate_collocation_points(10000, L, T)

initial_conditions = ["sin", "step", "gaussian"]

# initial_conditions = ["sin"]
boundary_conditions = ["dirichlet", "neumann", "periodic"]
results_base = []
results_import = []
results_gauss = []
results_gauss_res = []

num_experiments = 20  # <-- adjust to desired experiment count

for ic in initial_conditions:
    for bc in boundary_conditions:
        print(f"\n==== Running experiments for IC: {ic}, BC: {bc} ====")

        val_losses_base = []
        val_losses_import = []
        val_losses_gauss = []
        val_losses_gauss_res = []

        threshold_base_val = []
        threshold_import_val = []
        threshold_gauss_val = []
        threshold_gauss_val_res = []

        base_all_scores = []
        import_all_scores = []
        output_all_scores = []
        res_all_scores = []

        x_bc = torch.linspace(0, L, N_bc).view(-1, 1).to(device)
        t_bc = torch.zeros_like(x_bc).to(device)
        u_bc = initial_condition(x_bc, ic)

        for exp in range(num_experiments):
            print(f"--- Experiment {exp+1}/{num_experiments} ---")

            #Base
            model_base = PINN(layers).to(device)
            optimizer = optim.Adam(model_base.parameters(), lr=1e-3)
            thresh_base, base_scores = train(model_base, optimizer, N_f,x_f, t_f, x_val, t_val, x_bc, t_bc, u_bc, epochs=epochs, threshold=threshold)
            val_losses_base.append(val_loss(model_base, x_val, t_val, x_bc, t_bc, u_bc).item())
            threshold_base_val.append(thresh_base)
            base_all_scores.append(base_scores)
            del model_base, optimizer
            gc.collect(); torch.cuda.empty_cache()

            #Importance
            model_import = PINN(layers).to(device)
            optimizer = optim.Adam(model_import.parameters(), lr=1e-3)
            thresh_import, import_scores = train_import(model_import, optimizer, N_f, x_f, t_f, x_val, t_val, x_bc, t_bc, u_bc, epochs=epochs, threshold=threshold)
            val_losses_import.append(val_loss(model_import, x_val, t_val, x_bc, t_bc, u_bc).item())
            threshold_import_val.append(thresh_import)
            import_all_scores.append(import_scores)
            del model_import, optimizer
            gc.collect(); torch.cuda.empty_cache()

            #GP Output
            model_GP = PINN(layers).to(device)
            optimizer = optim.Adam(model_GP.parameters(), lr=1e-3)
            thresh_gp, output_scores = train_GP(model_GP, optimizer, N_f, x_f, t_f, x_val, t_val, x_bc, t_bc, u_bc, epochs=epochs, resample_every=100, threshold=threshold)
            val_losses_gauss.append(val_loss(model_GP, x_val, t_val, x_bc, t_bc, u_bc).item())
            threshold_gauss_val.append(thresh_gp)
            output_all_scores.append(output_scores)
            del model_GP, optimizer
            gc.collect(); torch.cuda.empty_cache()

            #GP Residual
            model_GP_res = PINN(layers).to(device)
            optimizer = optim.Adam(model_GP_res.parameters(), lr=1e-3)
            thresh_gp_res, res_scores = train_GP_res(model_GP_res, optimizer, N_f, x_f, t_f, x_val, t_val, x_bc, t_bc, u_bc, epochs=epochs, resample_every=100, threshold=threshold)
            val_losses_gauss_res.append(val_loss(model_GP_res, x_val, t_val, x_bc, t_bc, u_bc).item())
            threshold_gauss_val_res.append(thresh_gp_res)
            res_all_scores.append(res_scores)
            del model_GP_res, optimizer
            gc.collect(); torch.cuda.empty_cache()

        avg_base, std_base = np.mean(val_losses_base), np.std(val_losses_base)
        avg_import, std_import = np.mean(val_losses_import), np.std(val_losses_import)
        avg_gauss, std_gauss = np.mean(val_losses_gauss), np.std(val_losses_gauss)
        avg_gauss_res, std_gauss_res = np.mean(val_losses_gauss_res), np.std(val_losses_gauss_res)

        avg_t_base, std_t_base = np.mean(threshold_base_val), np.std(threshold_base_val)
        avg_t_import, std_t_import = np.mean(threshold_import_val), np.std(threshold_import_val)
        avg_t_gauss, std_t_gauss = np.mean(threshold_gauss_val), np.std(threshold_gauss_val)
        avg_t_gauss_res, std_t_gauss_res = np.mean(threshold_gauss_val_res), np.std(threshold_gauss_val_res)

        print(f"Average Validation Loss (Base):   {avg_base:.6f} ± {std_base:.6f}")
        print(f"Average Validation Loss (Import): {avg_import:.6f} ± {std_import:.6f}")
        print(f"Average Validation Loss (GP Out): {avg_gauss:.6f} ± {std_gauss:.6f}")
        print(f"Average Validation Loss (GP Res): {avg_gauss_res:.6f} ± {std_gauss_res:.6f}")

        results_base.append([ic, bc, avg_base, std_base, avg_t_base, std_t_base])
        results_import.append([ic, bc, avg_import, std_import, avg_t_import, std_t_import])
        results_gauss.append([ic, bc, avg_gauss, std_gauss, avg_t_gauss, std_t_gauss])
        results_gauss_res.append([ic, bc, avg_gauss_res, std_gauss_res, avg_t_gauss_res, std_t_gauss_res])

        # === Compute Epoch-wise Means and STDs ===
        # epochs_range = [i * 500 for i in range(len(base_all_scores[0]))]

        # base_all_scores_np = np.stack(base_all_scores)
        # import_all_scores_np = np.stack(import_all_scores)
        # output_all_scores_np = np.stack(output_all_scores)
        # res_all_scores_np = np.stack(res_all_scores)

        # base_mean_curve = np.mean(base_all_scores_np, axis=0)
        # base_std_curve = np.std(base_all_scores_np, axis=0)

        # import_mean_curve = np.mean(import_all_scores_np, axis=0)
        # import_std_curve = np.std(import_all_scores_np, axis=0)

        # output_mean_curve = np.mean(output_all_scores_np, axis=0)
        # output_std_curve = np.std(output_all_scores_np, axis=0)

        # res_mean_curve = np.mean(res_all_scores_np, axis=0)
        # res_std_curve = np.std(res_all_scores_np, axis=0)

        # # === Plot Mean ± Std ===
        # plt.figure(figsize=(10, 6))

        # plt.plot(epochs_range, base_mean_curve, label='Base PINN')
        # plt.fill_between(epochs_range, base_mean_curve - base_std_curve, base_mean_curve + base_std_curve, alpha=0.2)

        # plt.plot(epochs_range, import_mean_curve, label='Importance Sampling')
        # plt.fill_between(epochs_range, import_mean_curve - import_std_curve, import_mean_curve + import_std_curve, alpha=0.2)

        # # plt.plot(epochs_range, output_mean_curve, label='GP Output')
        # # plt.fill_between(epochs_range, output_mean_curve - output_std_curve, output_mean_curve + output_std_curve, alpha=0.2)

        # plt.plot(epochs_range, res_mean_curve, label='GP Residual')
        # plt.fill_between(epochs_range, res_mean_curve - res_std_curve, res_mean_curve + res_std_curve, alpha=0.2)

        # plt.xlabel('Epoch')
        # plt.ylabel('Mean Validation Loss')
        # plt.title(f'Mean Validation Loss ± Std vs Epochs\nIC: {ic}, BC: {bc}')
        # plt.legend()
        # plt.grid(True)
        # plt.tight_layout()

        # # Save figure
        # # filename = f"/mnt/data/val_loss_plot_ic_{ic}_bc_{bc}.png"
        # # plt.savefig(filename)
        # plt.show()

        


==== Running experiments for IC: sin, BC: dirichlet ====
--- Experiment 1/20 ---


/home/cadmin/anaconda3/envs/Data_Science_V2/lib/python3.11/site-packages/torch/autograd/graph.py:823: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:180.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 0, Loss: 0.3819037079811096
Epoch 500, Loss: 0.007066420279443264
Epoch 1000, Loss: 0.003041023388504982
Epoch 1500, Loss: 0.0016339098801836371
Epoch 2000, Loss: 0.0013910778798162937
Threshold reach at: 2321
Val loss: tensor(0.0010, device='cuda:0', grad_fn=<AddBackward0>)
Epoch 2500, Loss: 0.0011884781997650862
Epoch 3000, Loss: 0.0006122581544332206
Epoch 3500, Loss: 0.0004267664044164121
Epoch 4000, Loss: 0.0007234815275296569
Epoch 4500, Loss: 0.0009468795033171773
Epoch 4999, Loss: 0.000689165957737714
Epoch 0, Loss: 0.6155887842178345
Epoch 500, Loss: 0.01669834554195404
Epoch 1000, Loss: 0.010408066213130951
Epoch 1500, Loss: 0.005557235796004534
Epoch 2000, Loss: 0.004058976657688618
Epoch 2500, Loss: 0.003212453331798315
Epoch 3000, Loss: 0.0027661207132041454
Epoch 3500, Loss: 0.002150405663996935
Epoch 4000, Loss: 0.0018732609460130334
Epoch 4500, Loss: 0.0019012223929166794
Threshold reach at: 4698
Val loss: tensor(0.0010, device='cuda:0', grad_fn=<AddBackward0>)
Ep

In [23]:
import pandas as pd

# Convert results to DataFrames
df_base = pd.DataFrame(results_base, columns=["Initial Condition", "Boundary Condition", "Avg Validation Loss", "Std Validation Loss", "Avg Threshold","Std Threshold"])
df_import = pd.DataFrame(results_import, columns=["Initial Condition", "Boundary Condition", "Avg Validation Loss", "Std Validation Loss", "Avg Threshold","Std Threshold"])
df_gauss = pd.DataFrame(results_gauss, columns=["Initial Condition", "Boundary Condition", "Avg Validation Loss", "Std Validation Loss", "Avg Threshold","Std Threshold"])
df_gauss_res = pd.DataFrame(results_gauss_res, columns=["Initial Condition", "Boundary Condition", "Avg Validation Loss", "Std Validation Loss", "Avg Threshold","Std Threshold"])
# # Save to CSV
df_base.to_csv("Results_Experiments/base_05_27_2025.csv", index=False)
df_import.to_csv("Results_Experiments/import_05_27_2025.csv", index=False)
df_gauss.to_csv("Results_Experiments/GP_05_27_2025.csv", index=False)
df_gauss_res.to_csv("Results_Experiments/GP_final_05_27_2025.csv", index=False)

In [24]:
# import pandas as pd
df_base = pd.read_csv("Results_Experiments/base_05_27_2025.csv")
df_import = pd.read_csv("Results_Experiments/import_05_27_2025.csv")
df_gauss = pd.read_csv("Results_Experiments/GP_05_27_2025.csv")
df_gauss_res = pd.read_csv("Results_Experiments/GP_final_05_27_2025.csv")


In [25]:
df_base 

,Initial Condition,Boundary Condition,Avg Validation Loss,Std Validation Loss,Avg Threshold,Std Threshold
0,sin,dirichlet,0.000650,0.000215,3124.7,780.638847
1,sin,neumann,0.000644,0.000199,3197.8,727.776587
2,sin,periodic,0.000660,0.000197,3273.0,880.022897
3,step,dirichlet,0.047884,0.000758,5000.0,0.000000
4,step,neumann,0.048095,0.000735,5000.0,0.000000
5,step,periodic,0.047940,0.000614,5000.0,0.000000
6,gaussian,dirichlet,0.017101,0.000478,5000.0,0.000000
7,gaussian,neumann,0.017061,0.000441,5000.0,0.000000
8,gaussian,periodic,0.016998,0.000273,5000.0,0.000000


In [26]:
df_import

,Initial Condition,Boundary Condition,Avg Validation Loss,Std Validation Loss,Avg Threshold,Std Threshold
0,sin,dirichlet,0.001443,0.000445,4425.5,752.103816
1,sin,neumann,0.001462,0.000426,4496.2,563.973545
2,sin,periodic,0.001437,0.000415,4626.7,431.268953
3,step,dirichlet,0.063162,0.001924,5000.0,0.000000
4,step,neumann,0.063659,0.001540,5000.0,0.000000
5,step,periodic,0.063368,0.001567,5000.0,0.000000
6,gaussian,dirichlet,0.025441,0.000819,5000.0,0.000000
7,gaussian,neumann,0.025574,0.001264,5000.0,0.000000
8,gaussian,periodic,0.025713,0.000797,5000.0,0.000000


In [27]:
df_gauss

,Initial Condition,Boundary Condition,Avg Validation Loss,Std Validation Loss,Avg Threshold,Std Threshold
0,sin,dirichlet,0.019498,0.014795,5000.0,0.0
1,sin,neumann,0.016104,0.010140,5000.0,0.0
2,sin,periodic,0.026916,0.020239,5000.0,0.0
3,step,dirichlet,0.057072,0.005609,5000.0,0.0
4,step,neumann,0.056300,0.001599,5000.0,0.0
5,step,periodic,0.055806,0.002336,5000.0,0.0
6,gaussian,dirichlet,0.022224,0.001295,5000.0,0.0
7,gaussian,neumann,0.022068,0.001193,5000.0,0.0
8,gaussian,periodic,0.023413,0.002294,5000.0,0.0


In [28]:
df_gauss_res

,Initial Condition,Boundary Condition,Avg Validation Loss,Std Validation Loss,Avg Threshold,Std Threshold
0,sin,dirichlet,0.000367,0.000089,2002.95,415.757198
1,sin,neumann,0.000405,0.000145,2183.90,565.794300
2,sin,periodic,0.000373,0.000090,2023.20,488.306215
3,step,dirichlet,0.053176,0.002083,5000.00,0.000000
4,step,neumann,0.054435,0.004868,5000.00,0.000000
5,step,periodic,0.056691,0.006769,5000.00,0.000000
6,gaussian,dirichlet,0.020037,0.001869,5000.00,0.000000
7,gaussian,neumann,0.020674,0.001464,5000.00,0.000000
8,gaussian,periodic,0.020281,0.001518,5000.00,0.000000
